In [2]:
from os import access
import xlwings as xw
from connect_to_dhan import Connection
from dhanhq import dhanhq
import numpy as np
import pandas as pd
from dhanhq import marketfeed
import yaml
import time



In [3]:

""" configuring the workbook and getting the security info
"""
# Load Excel file
excel_file = 'DhanTrading.xlsx'
workbook = xw.Book(excel_file)
TradeSheet = workbook.sheets['Trade']
OptionsLookUp = workbook.sheets['OptionsLookUp'] 
IndexLookup = workbook.sheets['IndexLookup']

In [4]:
def option_age(x):  # This function will classify the options strike if it belongs to Current series, Next series or Far next.
    if x == 1.0:
        return 'C'  # Current series
    elif x == 2.0:
        return 'N'  # Next Series
    else:
        return 'F'  # Far next series

def connect_to_dhan():
    config_file_path = r".\\config\\config.yaml" #This has the API key and client id
    with open(config_file_path,'r') as config:
        api_config = yaml.safe_load(config)
    clinet_id = api_config.get('api_config')[1]
    access_token = api_config.get('api_config')[2]

    # Establish connection to Dhan
    try:
        DhanConnector = Connection(clinet_id, access_token)
        ConnectionObject = DhanConnector.connect_dhan()
        dhan = ConnectionObject['conn']
    except Exception as e :
        raise ConnectionError(f"Can't connect {e}")  

    return({"connection":dhan,
            "client_id":clinet_id,
            "access_token":access_token})    

Configure the sheets

In [5]:
def initial_sheet_config():
    connections__= connect_to_dhan() #returned as dictionary but accessed like a list
    dhan = connections__['connection']
    client_id = connections__['client_id']
    access_token = connections__['access_token']
    # Load the latest keys and scripts metadata automatically
    security_list = dhan.fetch_security_list("compact")
    #--------------------------------------------------------------------------------------------
    # index
    #--------------------------------------------------------------------------------------------
    mcx_fut =  security_list[(security_list['SEM_EXM_EXCH_ID']=='MCX') & 
                                (security_list['SM_SYMBOL_NAME']=='CRUDEOIL') & 
                                (security_list['SEM_INSTRUMENT_NAME']=='FUTCOM')]
    index = security_list[(security_list['SEM_INSTRUMENT_NAME'] == 'INDEX') & (
        (security_list['SEM_TRADING_SYMBOL']=='BANKNIFTY') |
        (security_list['SEM_TRADING_SYMBOL']=='NIFTY')) & (security_list['SEM_SEGMENT']=='I')]
    filtered_df_index = pd.concat([mcx_fut,index])
    filtered_df_index = filtered_df_index.copy()
    filtered_df_index['SEM_TRADING_SYMBOL']= [x[0] for x in filtered_df_index.SEM_TRADING_SYMBOL.str.split('-')]

    filtered_df_index['Rank'] = filtered_df_index.groupby('SEM_EXM_EXCH_ID')['SEM_EXPIRY_DATE'].rank(method='dense', ascending=True)
    filtered_df_index = filtered_df_index[(filtered_df_index['Rank']==1.0) | (np.isnan(filtered_df_index['Rank']))]
    IndexLookup.clear()
    IndexLookup.range('A1').options(index=False).value = filtered_df_index[['SEM_TRADING_SYMBOL','SEM_SMST_SECURITY_ID']]
    #--------------------------------------------------------------------------------------------
    # Filter for only NIFTY and BANKNIFTY OPTIONS and MCX
    #--------------------------------------------------------------------------------------------
    NSE = security_list[(
        (security_list['SEM_EXM_EXCH_ID'] == 'NSE') & 
        (security_list['SEM_INSTRUMENT_NAME'] == 'OPTIDX') & 
        ((security_list['SEM_TRADING_SYMBOL'].str.startswith('BANKNIFTY')) | 
            (security_list['SEM_TRADING_SYMBOL'].str.startswith('NIFTY')))
        &
        (security_list['SEM_TRADING_SYMBOL'].str.startswith('NIFTYNXT50') == False)
    )]    
    MCX = security_list[(security_list['SEM_EXM_EXCH_ID'] == 'MCX') & 
                        (security_list['SEM_INSTRUMENT_NAME'] == 'OPTFUT') & 
                        (security_list['SM_SYMBOL_NAME'] == 'CRUDEOIL')]

    filtered_df = pd.concat([NSE, MCX])
    filtered_df = filtered_df.copy()  

    # Below logic will help us to classify the Series type {C - Current, N - Next and F - Future}
    filtered_df['SEM_EXPIRY_DATE_CUSTOM'] = np.nan
    filtered_df['SEM_EXPIRY_DATE_CUSTOM'] = pd.to_datetime(filtered_df.SEM_EXPIRY_DATE)
    filtered_df['SEM_UNDERLYING'] = [x[0] for x in filtered_df.SEM_CUSTOM_SYMBOL.str.split(' ')]
    filtered_df['Rank'] = filtered_df.groupby('SEM_UNDERLYING')['SEM_EXPIRY_DATE_CUSTOM'].rank(method='dense', ascending=True)
    filtered_df['Series'] = filtered_df['Rank'].apply(option_age)

    filtered_df = filtered_df[filtered_df['Series'] == 'C'].copy(deep=True)
    filtered_df['SEM_STRIKE_PRICE'] = filtered_df['SEM_STRIKE_PRICE'].astype(int)
    filtered_df = filtered_df[['SEM_SMST_SECURITY_ID','SEM_UNDERLYING','SEM_OPTION_TYPE','SEM_STRIKE_PRICE']].copy(deep=True)
    filtered_df['Series'] = filtered_df['SEM_UNDERLYING']+"_"+filtered_df['SEM_OPTION_TYPE']+"_"+filtered_df['SEM_STRIKE_PRICE'].astype(str)
    OptionsLookUp.clear()
    OptionsLookUp.range('A1').options(index=False).value = filtered_df[['Series','SEM_SMST_SECURITY_ID']]

In [ ]:
def refresh_instruments():
    index_key = TradeSheet.range("IndexKey").value
    atm_key = TradeSheet.range("ATM_KEY").value
    itm_1_key = TradeSheet.range("ITM_ONE_KEY").value
    itm_2_key = TradeSheet.range("ITM_TWO_KEY").value
    itm_3_key = TradeSheet.range("ITM_THREE_KEY").value
    return([index_key,atm_key,itm_1_key,itm_2_key,itm_3_key])